# Многоуровневая оптимизация
## Синтетический эксперемент: Минимизация функции Розенброка с помощью:

Low-fidelity (быстрая, шумная, приближённая)

High-fidelity (медленная, точная)

Использование стратегии вроде MFBO (multi-fidelity Bayesian optimization)

Графики затрат / точности / итераций

## Реальный эксперемент (только финальные графики или табличные результаты)

Приоритезация стека расчетов ИМА из Оптима